In [ ]:
import openseespy.opensees as ops
import os as os
import numpy as np
import matplotlib.pyplot as plt
import time as tmes
import numpy.matlib as mtlib

def Two_d_frame(ct,NStory,NBay,Fy,Es,nu,MyCol,MyBeam,data_set):

    LCol=14*12 #in
    LBeam=24*12 #in
    NStory=NStory # number of stories above ground level -------------- you can change this.
    NBay= NBay
    ops.wipe()
    ops.model('basic', '-ndm', 2, '-ndf', 3)

    # Create nodes
    iSupportNode=[]
    roof_node=[]
    roof_below_node=[]
    roof_below2_node=[]
    for level in range(NStory+1):
        Y=(level)*LCol
        for pier in range(NBay+1):
            X=pier*LBeam
            nodeID=level*10+pier
            ops.node(nodeID,X,Y)
            if level==0:
                iSupportNode=np.append(iSupportNode,nodeID)
            elif level==NStory:
                roof_node=np.append(roof_node,nodeID)
            elif level==NStory-1:
                roof_below_node=np.append(roof_below_node,nodeID)
            elif level==NStory-2:
                roof_below2_node=np.append(roof_below2_node,nodeID)
    print(roof_node)
    print(roof_below_node)
  


    # BOUNDARY CONDITIONS
    ops.fixY(0.0, 1, 1, 0)# pin all Y=0.0 nodes


    # Parameters needed for displacement control
    IDctrlNode=(NStory+1)*10+1
    IDctrlDOF=1
    LBuilding=NStory*LCol


    # Define ELEMENTS & SECTIONS 
    ColSecTag=1
    ColMatTagFlex=2
    ColMatTagAxial=3
    BeamSecTag=4
    BeamMatTagFlex=5
    BeamMatTagAxial=6


    # Define MATERIAL properties
    Fy =Fy #ksi
    Es=Es #ksi
    nu=nu
    Gs=Es/(2*(1+nu))


    # Column section
    AgCol =33.5 #in^2
    IzCol=4090 #in^4
    EICol=Es*IzCol
    EACol=Es*AgCol
    MyCol=MyCol #kip-in yield Moment
    PhiYCol=0.25e-3 #/in #yield curvature
    PhiYCol=MyCol/EICol
    EIColCrack=MyCol/PhiYCol
    b=0.01 #strain hardening ratio
    

    ops.uniaxialMaterial("Steel01", ColMatTagFlex, MyCol, EIColCrack,b)# bilinear behavior for flexure
    ops.uniaxialMaterial("Elastic",ColMatTagAxial, EACol)# this is not used as a material, this is an axial-force-strain response
    ops.section("Aggregator",ColSecTag,ColMatTagAxial, "P",ColMatTagFlex, "Mz") #combine axial and flexural behavior into one section (no P-M interaction here)


    # Column section
    AgBeam=27.7# cross-sectional area
    IzBeam= 2700# moment of Inertia
    EIBeam = Es*IzBeam# EI, for moment-curvature relationship
    EABeam = Es*AgBeam# EA, for axial-force-strain relationship
    MyBeam = MyBeam# yield moment
    PhiYBeam = 0.25e-3# yield curvature
    PhiYBeam = MyBeam/EIBeam# yield curvature
    EIBeamCrack = MyBeam/PhiYBeam# cracked section inertia
    b=0.01 # strain-hardening ratio (ratio between post-yield tangent and initial elastic tangent)

    ops.uniaxialMaterial("Steel01", BeamMatTagFlex, MyBeam, EIBeamCrack, b)# bilinear behavior for flexure
    ops.uniaxialMaterial("Elastic", BeamMatTagAxial, EABeam)# this is not used as a material, this is an axial-force-strain response
    ops.section("Aggregator", BeamSecTag, BeamMatTagAxial, "P", BeamMatTagFlex,"Mz")# combine axial and flexural behavior into one section (no P-M interaction here)



    # define ELEMENTS
    # up geometric transformations of element
    #   separate columns and beams, in case of P-Delta analysis for columns
    IDColTransf=1 # all columns
    IDBeamTransf=2 # all beams
    ColTransfType='Linear'# options, Linear PDelta Corotational 
    ops.geomTransf(ColTransfType,IDColTransf)# only columns can have PDelta effects (gravity effects)
    ops.geomTransf('Linear',IDBeamTransf)


    npts=5 # number of Gauss integration points for nonlinear curvature distribution-- np=2 for linear distribution ok
    # columns
    N0col=100 # column element numbers


    for level in range(NStory):
        for pier in range(NBay+1):
            nodeI=level*10+pier
            nodeJ=(level+1)*10+pier
            elemID=N0col+level*10+pier
            ops.element('nonlinearBeamColumn', elemID,nodeI,nodeJ,npts,ColSecTag, IDColTransf)#columns
    roof_col=elemID       

    N0beam=200 # beam element numbers
    

    for level in range(1,NStory+1):
        for bay in range(NBay):
            nodeI=level*10+bay
            nodeJ=level*10+bay+1
            elemID=N0beam+level*10+bay
            ops.element('nonlinearBeamColumn', elemID,nodeI,nodeJ,npts,BeamSecTag, IDBeamTransf)#columns
    roof_beam=elemID

    # Define GRAVITY LOADS, weight and masses
    # calculate dead load of frame, assume this to be an internal frame (do LL in a similar manner)
    # calculate distributed weight along the beam length
    GammaConcrete =150*5.787037037037037e-07 #pcf to kip/inc3   # Reinforced-Concrete floor slabs
    Tslab =6 # 6-inch slab
    Lslab =2*LBeam/2 # assume slab extends a distance of $LBeam1/2 in/out of plane
    Qslab =GammaConcrete*Tslab*Lslab
    QBeam =94*(10**-3)/12# W-section weight per length lb/ft -> kip/inch
    QdlBeam =Qslab + QBeam # dead load distributed along beam.
    QdlCol =114*(10**-3)/12# W-section weight per length
    WeightCol=QdlCol*LCol # total Column weight
    WeightBeam= QdlBeam*LBeam# total Beam weight

    # iFloorWeight=[]
    # WeightTotal=0.0

    ops.timeSeries("Linear", 1000)
    ops.pattern("Plain", 101, 1000)
    g=386.4


    for level in range(1,NStory+1):

        FloorWeight=0.0
        if level == NStory+1:
            ColWeightFact=1# one column in top story
        else:
            ColWeightFact=2		# two columns elsewhere
        pier=0  
        for pier in range(NBay+1):
            if pier == 0 or pier == NBay:
                BeamWeightFact=1	# one beam at exterior nodes
            else:
                BeamWeightFact=2	# two beams elewhere

            WeightNode=ColWeightFact*WeightCol/2 + BeamWeightFact*WeightBeam/2
            MassNode=WeightNode/g
  
            nodeID=level*10+pier

            ops.mass(nodeID,MassNode, 0.0, 0.0)# define mass
            ops.load(nodeID,0.0,-WeightNode,0.0)
            


    ops.constraints('Plain')
    ops.numberer('RCM')
    ops.system('BandGeneral')
    ops.test('NormDispIncr', 1e-8, 6)
    ops.algorithm('Newton')
    ops.integrator('LoadControl', 0.1)
    ops.analysis('Static')
    ops.analyze(10)
    ops.loadConst('-time', 0.0)
    # ops.wipeAnalysis()
    
    
    
    Tol = 1e-8
    maxNumIter = 2
    GMdirection = 1
#     GMfact = 1.5
#     GMfatt = g*GMfact
    DtAnalysis = 0.01 # time-step Dt for lateral analysis
    TmaxAnalysis = 10.0 # maximum duration of ground-motion analysis


    Lambda = ops.eigen(3)
    Omega = Lambda[0]**0.5

    xDamp=0.05 # damping ratio
    MpropSwitch=1.0
    KcurrSwitch=0.0
    KcommSwitch=1.0
    KinitSwitch=0.0
    nEigenI=0 # mode 1
    nEigenJ=2 # mode 3
    Lambda = ops.eigen(3)
    
    lambdaI=Lambda[nEigenI] # eigenvalue mode i
    lambdaJ=Lambda[nEigenJ]
    omegaI=lambdaI**0.5
    omegaJ=lambdaJ**0.5
    alphaM=MpropSwitch*xDamp*(2*omegaI*omegaJ)/(omegaI+omegaJ)# M-prop. damping; D = alphaM*M
    betaKcurr=KcurrSwitch*2*xDamp/(omegaI+omegaJ)# current-K;      +beatKcurr*KCurrent
    betaKcomm=KcommSwitch*2*xDamp/(omegaI+omegaJ)# last-committed K;   +betaKcomm*KlastCommitt
    betaKinit=KinitSwitch*2*xDamp/(omegaI+omegaJ)# initial-K;     +beatKinit*Kini

    # define damping
    ops.rayleigh(alphaM, betaKcurr, betaKinit,betaKcomm)

    
    time_history_temp= np.loadtxt('Time_eq.txt')
    nPts=len(time_history_temp)
    dt=time_history_temp[1]-time_history_temp[0]

    
    if data_set=='training':
#         print('training')
        load_history="load_files/Load.{}.txt".format(ct)
    elif data_set=='IDA':
#         print('IDA')
        load_history="load_files/Load_IDA.{}.txt".format(ct)
    else:
#         print('Real EQ')
        load_history="Load_nep1.txt"

    # Uniform EXCITATION: acceleration input
    IDloadTag = 400 # load tag
    ops.timeSeries('Path', 20000, '-dt', dt, '-filePath', load_history, '-factor', 1.0)
    ops.pattern('UniformExcitation', IDloadTag, GMdirection, '-accel', 20000) 

    ops.constraints('Transformation')
    ops.numberer('RCM')
    ops.system('BandGeneral')
 

    tCurrent = ops.getTime()
    

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 3:'EnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 3:'ModifiedNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}
#     test = {1:'NormDispIncr'}#, 2: 'RelativeEnergyIncr', 3:'EnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
#     algorithm = {1:'ModifiedNewton'}#, 2: 'SecantNewton' , 3:'ModifiedNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}
    
    tFinal = nPts*dt


#     tFinal = 60.0
    time = [tCurrent]
    u0=[0.0]
    u1 = [0.0]
    u2 = [0.0]
    u3 = [0.0]
    u4 = [0.0]
    u5 = [0.0]
    u6 = [0.0]
    u7 = [0.0]
    u8 = [0.0]
    u9 = [0.0]
#     ok = 0
    maxNumIter=10
    start = tmes.time()
    elapsed_time_fl=0
    tstng=0
    ok=0
    while tCurrent < tFinal:
        if tstng==37 and ok!=0:
            break 
        ok=0
        for i in test:
#             if tstng==37 and ok!=0:
#                 break

            for j in algorithm:
                hours, rem = divmod(tmes.time() - start, 3600)
                minutes, seconds = divmod(rem, 60)
#                 print(minutes,seconds,algorithm[j],ok)
            
                if minutes>1:
                    tstng=37
               
                if j < 4:
        
                    ops.algorithm(algorithm[j], '-initial')

                else:
                    ops.algorithm(algorithm[j])
                while ok == 0 and tCurrent < tFinal:
#                     hours, rem = divmod(tmes.time() - start, 3600)
#                     minutes, seconds = divmod(rem, 60)
#                     print(minutes,seconds)
                    if tstng==37:
                        break
#                     if minutes>=1:
#                         tstng=37
#                         break

                    ops.test(test[i], Tol, maxNumIter)        
                    NewmarkGamma = 0.5
                    NewmarkBeta = 0.25
                    ops.integrator('Newmark', NewmarkGamma, NewmarkBeta)
                    ops.analysis('Transient')
                    ok = ops.analyze(1, dt/2)
#                     print(ok)
#                     print(elapsed_time_fl)
                    if ok == 0 :
                        tCurrent = ops.getTime() 
                        
                        
                        
#                         print(tCurrent)
                        time.append(tCurrent)
                        u0.append(ops.nodeDisp(int(0),1))
        
                        u1.append(ops.nodeDisp(int(roof_node[0]),1))
                        u2.append(ops.nodeDisp(int(roof_node[1]),1))
                        u3.append(ops.nodeDisp(int(roof_node[2]),1))
                    
                        u4.append(ops.nodeDisp(int(roof_below_node[0]),1))
                        u5.append(ops.nodeDisp(int(roof_below_node[1]),1))
                        u6.append(ops.nodeDisp(int(roof_below_node[2]),1))
                        
                        u7.append(ops.nodeDisp(int(roof_below2_node[0]),1))
                        u8.append(ops.nodeDisp(int(roof_below2_node[1]),1))
                        u9.append(ops.nodeDisp(int(roof_below2_node[2]),1))
#                     print(test[i], algorithm[j], 'tCurrent=', tCurrent)



#     u_max=np.max(np.max(np.abs(np.array(u1))),np.max(np.abs(np.array(u2))),np.max(np.abs(np.array(u3))),axis=0)
#     u_max=np.max(np.abs(np.array(u1)))
    u_max=np.max(np.abs(np.array(u7)))
#     u_max=np.max(np.abs(np.array(u1)-np.array(u4)))
#     import matplotlib.pyplot as plt
#     plt.figure(figsize=(8,8))
#     plt.plot(time, u1)
#     plt.plot(time, u4)
#     plt.plot(time, u7)
#     plt.plot(time, np.array(u1)-np.array(u4))
#     plt.ylabel('Horizontal Displacement of node 3 (in)')
#     plt.xlabel('Time (s)')
#     plt.savefig('Horizontal Disp at Node 3 vs time.jpeg')
#     plt.show()
    if tstng==37:
        return np.array([999999,MyCol,MyBeam,ct]) 
    else:
        return np.array([u_max,MyCol,MyBeam,ct]) 
    ops.wipe()

In [ ]:
os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))

Ndof=int(np.loadtxt('Ndof.txt'))
stf=np.loadtxt('Stiffnes{}Dof.txt'.format(Ndof)) # For training
# dta=Two_d_frame(ct,3,3,6.0,29000,0.3,stf[ct,0],stf[ct,1],'training')
from joblib import Parallel, delayed
results=Parallel(n_jobs=120)(delayed(Two_d_frame)(ct,3,3,6.0,stf[ct,0],0.3,stf[ct,1],stf[ct,2],'training') for ct in np.arange(0,len(stf)))
res=np.array(results[:])
os.chdir(os.path.expanduser('~/Dropbox/Research/Building ML/Paper 2/'))
load_coeff=np.loadtxt('wv_coeff.txt')
u_max=np.expand_dims(res[:,0],axis=1)
Dta_set=np.concatenate((load_coeff,stf,u_max),axis=1)
ind=np.where(Dta_set[:,-1]>1000)
Dta_set_temp=np.delete(Dta_set,ind,axis=0)
Dta_set=Dta_set_temp
#np.save('Dta_2Dframe_0.npy',Dta_set)


In [ ]:
os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))
s=np.loadtxt('load_files/Load.0.txt')
s.shape
t=np.loadtxt('Time_eq.txt')
t.shape

In [ ]:
# Dta_set=np.delete(Dta_set,192,axis=1)
# np.save('Dta_2Dframe.npy',Dta_set)


In [ ]:
# ct=3
# os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))
# Ndof=int(np.loadtxt('Ndof.txt'))
# stf=np.loadtxt('Stiffnes{}Dof.txt'.format(Ndof)) # For training
# # # Dta_set.shape
# # os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))
# Two_d_frame(ct,3,3,6.0,stf[ct,0],0.3,stf[ct,1],stf[ct,2],'training')

In [ ]:
os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))

Ndof=int(np.loadtxt('Ndof.txt'))
stf=np.loadtxt("Stiffnes{}Dof_1eq.txt".format(Ndof)) # For training
# dta=Two_d_frame(ct,3,3,6.0,29000,0.3,stf[ct,0],stf[ct,1],'training')
from joblib import Parallel, delayed
results=Parallel(n_jobs=60)(delayed(Two_d_frame)(ct,3,3,6.0,stf[ct,0],0.3,stf[ct,1],stf[ct,2],'Real_EQ') for ct in np.arange(0,len(stf)))
res=np.array(results[:])
os.chdir(os.path.expanduser('~/Dropbox/Research/Building ML/Paper 2/'))
load_c=np.expand_dims(np.loadtxt('coeff_real_eq.txt'),axis=1)
load_coeff=mtlib.repmat(load_c.T,len(stf),1)
u_max=np.expand_dims(res[:,0],axis=1)
Dta_set=np.concatenate((load_coeff,stf,u_max),axis=1)
ind=np.where(Dta_set[:,-1]>1000)
Dta_set_temp=np.delete(Dta_set,ind,axis=0)
Dta_set=Dta_set_temp
np.save('Dta_2Dframe_1eq.npy',Dta_set)

In [ ]:
# Dta_set=np.delete(Dta_set,192,axis=1)
# np.save('Dta_2Dframe_1eq.npy',Dta_set)

In [ ]:
# s=np.load('Dta_2Dframe.npy')
# plt.hist(s[:,-1],density=True)
# plt.hist(Dta_set[:,-1],density=True)

# os.chdir(os.path.expanduser('~/OpenSees/Opensees_ml_runs/'))
# ct=2
# Two_d_frame(ct,3,3,5.0,29000,0.3,2e4,1.5e4,'REAL_EQ')
# Two_d_frame(ct,3,3,5.0,29000,0.3,3e4,0.5e4,'REAL_EQ')

In [ ]:
# plt.hist(u_max)
# Ndof=int(np.loadtxt('Ndof.txt'))
# stf=np.loadtxt('Stiffnes{}Dof.txt'.format(Ndof)) # For training
# ct=15a
# dta=Two_d_frame(ct,3,3,6.0,29000,0.3,1.7e4,1.2e4,'training')

In [ ]:

# Dta_set=np.concatenate((Dta_set_tmp,res[:,0].squeeze),axis=0)

In [ ]:
# from matplotlib.pyplot import *
# load_history=np.loadtxt("load_files/Load.{}.txt".format(17))
# plot(load_history)
# xlim([0,1500])

In [ ]:


# u1=np.array(u1)
# u0=np.array(u0)
# plt.plot(u1)


In [ ]:
# s=
# s.shape

In [ ]:
# results

In [ ]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# os.chdir(os.path.expanduser('/home/sid/Dropbox/Research/Building ML/Paper 2/'))
# s=np.loadtxt('Tcl codes 2d frame/Data/sid.out')
# plt.plot(s[9:,0],u1)
# plt.plot(s[:,0],s[:,1]-s[8,1])


In [ ]:
# import openseespy.opensees as ops
# import numpy as np
# LCol=14*12 #in
# LBeam=24*12 #in
# NStory= 3 # number of stories above ground level -------------- you can change this.
# NBay= 3
# ops.wipe()
# ops.model('basic', '-ndm', 2, '-ndf', 3)

# # Create nodes
# iSupportNode=[]
# for level in range(NStory+1):
#     Y=(level)*LCol
#     for pier in range(NBay+1):
#         X=pier*LBeam
#         nodeID=level*10+pier
#         ops.node(nodeID,X,Y)
#         if level==0:
#             iSupportNode=np.append(iSupportNode,nodeID)
# #         print(nodeID,X,Y)
# # print(iSupportNode)


# # BOUNDARY CONDITIONS
# ops.fixY(0.0, 1, 1, 0)# pin all Y=0.0 nodes


# # Parameters needed for displacement control
# IDctrlNode=(NStory+1)*10+1
# IDctrlDOF=1
# LBuilding=NStory*LCol


# # Define ELEMENTS & SECTIONS 
# ColSecTag=1
# ColMatTagFlex=2
# ColMatTagAxial=3
# BeamSecTag=4
# BeamMatTagFlex=5
# BeamMatTagAxial=6


# # Define MATERIAL properties
# Fy =6.0 #ksi
# Es=29000 #ksi
# nu=0.3
# Gs=Es/(2*(1+nu))


# # Column section
# AgCol =33.5 #in^2
# IzCol=4090 #in^4
# EICol=Es*IzCol
# EACol=Es*AgCol
# MyCol=2e4 #kip-in yield Moment
# PhiYCol=0.25e-3 #/in #yield curvature
# PhiYCol=MyCol/EICol
# EIColCrack=MyCol/PhiYCol
# b=0.01 #strain hardening ratio
# print(AgCol,IzCol,MyCol)

# ops.uniaxialMaterial("Steel01", ColMatTagFlex, MyCol, EIColCrack,b)# bilinear behavior for flexure
# ops.uniaxialMaterial("Elastic",ColMatTagAxial, EACol)# this is not used as a material, this is an axial-force-strain response
# ops.section("Aggregator",ColSecTag,ColMatTagAxial, "P",ColMatTagFlex, "Mz") #combine axial and flexural behavior into one section (no P-M interaction here)


# # Column section
# AgBeam=27.7# cross-sectional area
# IzBeam= 2700# moment of Inertia
# EIBeam = Es*IzBeam# EI, for moment-curvature relationship
# EABeam = Es*AgBeam# EA, for axial-force-strain relationship
# MyBeam = 1.5e4# yield moment
# PhiYBeam = 0.25e-3# yield curvature
# PhiYBeam = MyBeam/EIBeam# yield curvature
# EIBeamCrack = MyBeam/PhiYBeam# cracked section inertia
# b=0.01 # strain-hardening ratio (ratio between post-yield tangent and initial elastic tangent)

# ops.uniaxialMaterial("Steel01", BeamMatTagFlex, MyBeam, EIBeamCrack, b)# bilinear behavior for flexure
# ops.uniaxialMaterial("Elastic", BeamMatTagAxial, EABeam)# this is not used as a material, this is an axial-force-strain response
# ops.section("Aggregator", BeamSecTag, BeamMatTagAxial, "P", BeamMatTagFlex,"Mz")# combine axial and flexural behavior into one section (no P-M interaction here)



# # define ELEMENTS
# # set up geometric transformations of element
# #   separate columns and beams, in case of P-Delta analysis for columns
# IDColTransf=1 # all columns
# IDBeamTransf=2 # all beams
# ColTransfType='Linear'# options, Linear PDelta Corotational 
# ops.geomTransf(ColTransfType,IDColTransf)# only columns can have PDelta effects (gravity effects)
# ops.geomTransf('Linear',IDBeamTransf)


# npts=5 # number of Gauss integration points for nonlinear curvature distribution-- np=2 for linear distribution ok
# # columns
# N0col=100 # column element numbers



# for level in range(NStory):
#     for pier in range(NBay+1):
#         nodeI=level*10+pier
#         nodeJ=(level+1)*10+pier
#         elemID=N0col+level*10+pier
#         ops.element('nonlinearBeamColumn', elemID,nodeI,nodeJ,np,ColSecTag, IDColTransf)#columns
#         print(elemID,nodeI,nodeJ,np,ColSecTag, IDColTransf)
# #         print(elemID,nodeI,nodeJ)

# N0beam=200 # beam element numbers


# for level in range(1,NStory+1):
#     for bay in range(NBay):
#         nodeI=level*10+bay
#         nodeJ=level*10+bay+1
#         elemID=N0beam+level*10+bay
#         ops.element('nonlinearBeamColumn', elemID,nodeI,nodeJ,npts,BeamSecTag, IDBeamTransf)#columns
#         print( elemID,nodeI,nodeJ,np,BeamSecTag, IDBeamTransf)
# #         print(elemID,nodeI,nodeJ)


#     if data_set=='training':
#         ops.recorder('Node', '-file', "output_files/Disp_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'disp')
#         ops.recorder('Node', '-file', "output_files/Vel_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'vel')
#         ops.recorder('Node', '-file', "output_files/Accel_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'accel')
#         ops.recorder('Element', '-file', "output_files/Def_eq_{}_col.txt".format(ct), '-ele', roof_col, 'deformation')
#         ops.recorder('Element', '-file', "output_files/Force_eq_{}_col.txt".format(ct), '-ele', roof_col, 'force')
#     elif data_set=='IDA':
#         ops.recorder('Node', '-file', "output_files/Disp_eq_IDA_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'disp')
#         ops.recorder('Node', '-file', "output_files/Vel_eq_IDA_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'vel')
#         ops.recorder('Node', '-file', "output_files/Accel_eq_IDA_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'accel')
#         ops.recorder('Element', '-file', "output_files/Def_eq_IDA_{}.txt".format(ct), '-ele', roof_col, 'deformation')
#         ops.recorder('Element', '-file', "output_files/Force_eq_IDA_{}.txt".format(ct), '-ele', roof_col, 'force')
#     else: 
# #         ops.recorder('Node', '-file', "output_files/Disp_real_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'disp')
# #         ops.recorder('Node', '-file', "output_files/Vel_real_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'vel')
# #         ops.recorder('Node', '-file', "output_files/Accel_real_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'accel')
# #         ops.recorder('Element', '-file', "output_files/Def_real_eq_{}.txt".format(ct), '-ele', roof_col, 'deformation')
# #         ops.recorder('Element', '-file', "output_files/Force_real_eq_{}.txt".format(ct), '-ele', roof_col, 'force')
#         ops.recorder('Node', '-file', "output_files/Disp_real_eq_{}.txt".format(ct), '-node', int(roof_node[0]), '-dof', 1, 'disp')
# #         ops.recorder('Node', '-file', "output_files/Vel_real_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'vel')
# #         ops.recorder('Node', '-file', "output_files/Accel_real_eq_{}.txt".format(ct), '-nodeRange', int(roof_node[0]),int(roof_node[-1]), '-dof', 1, 'accel')
# #         ops.recorder('Element', '-file', "output_files/Def_real_eq_{}.txt".format(ct), '-ele', roof_col, 'deformation')
# # Define GRAVITY LOADS, weight and masses
# # calculate dead load of frame, assume this to be an internal frame (do LL in a similar manner)
# # calculate distributed weight along the beam length
# GammaConcrete =150*5.787037037037037e-07 #pcf to kip/inc3   # Reinforced-Concrete floor slabs
# Tslab =6 # 6-inch slab
# Lslab =2*LBeam/2 # assume slab extends a distance of $LBeam1/2 in/out of plane
# Qslab =GammaConcrete*Tslab*Lslab
# QBeam =94*(10**-3)/12# W-section weight per length lb/ft -> kip/inch
# QdlBeam =Qslab + QBeam # dead load distributed along beam.
# QdlCol =114*(10**-3)/12# W-section weight per length
# WeightCol=QdlCol*LCol # total Column weight
# WeightBeam= QdlBeam*LBeam# total Beam weight

# # iFloorWeight=[]
# # WeightTotal=0.0

# ops.timeSeries("Linear", 1000)
# ops.pattern("Plain", 101, 1000)
# g=386.4




# for level in range(1,NStory+1):

#     FloorWeight=0.0
#     if level == NStory+1:
#         ColWeightFact=1# one column in top story
#     else:
#         ColWeightFact=2		# two columns elsewhere
#     pier=0  
#     for pier in range(NBay+1):
#         if pier == 0 or pier == NBay:
#             BeamWeightFact=1	# one beam at exterior nodes
#         else:
#             BeamWeightFact=2	# two beams elewhere

#         WeightNode=ColWeightFact*WeightCol/2 + BeamWeightFact*WeightBeam/2
#         MassNode=WeightNode/g
# #         print(MassNode)
#         nodeID=level*10+pier
        
#         ops.mass(nodeID,MassNode, 0.0, 0.0)# define mass
#         ops.load(nodeID,0.0,-WeightNode,0.0)
#         print(nodeID,MassNode, 0.0, 0.0)

# ops.recorder('Node', '-file', "sid.txt", '-node', 10, '-dof', 2, 'disp')

# ops.constraints('Plain')
# ops.numberer('RCM')
# ops.system('BandGeneral')
# ops.test('NormDispIncr', 1e-8, 6)
# ops.algorithm('Newton')
# ops.integrator('LoadControl', 0.1)
# ops.analysis('Static')
# ops.analyze(10)
# ops.loadConst('-time', 0.0)
# # ops.wipeAnalysis()

In [ ]:
# import math
# Tol = 1e-8
# maxNumIter = 10
# GMdirection = 1
# GMfact = 1.5
# GMfatt = g*GMfact
# DtAnalysis = 0.01 # time-step Dt for lateral analysis
# TmaxAnalysis = 10.0 # maximum duration of ground-motion analysis


# # Lambda = ops.eigen('-fullGenLapack', 1) # eigenvalue mode 1
# # Lambda = ops.eigen('-fullGenLapack', 1)
# Lambda = ops.eigen(3)
# Omega = Lambda[0]**0.5
# betaKcomm = 2 * (0.02/Omega)

# xDamp = 0.02# 2% damping ratio
# alphaM = 0.0# M-prop. damping; D = alphaM*M	
# betaKcurr = 0.0# K-proportional damping;      +beatKcurr*KCurrent
# betaKinit = 0.0 # initial-stiffness proportional damping      +beatKinit*Kini

# ops.rayleigh(alphaM,betaKcurr, betaKinit, betaKcomm) # RAYLEIGH damping

# # Set some parameters
# record = 'H-E12140'

# import ReadRecord
# # Permform the conversion from SMD record to OpenSees record
# dt, nPts = ReadRecord.ReadRecord(record+'.at2', record+'.dat')
# #print(dt, nPts)

# # Uniform EXCITATION: acceleration input
# IDloadTag = 400 # load tag
# ops.timeSeries('Path', 20000, '-dt', dt, '-filePath', record+'.dat', '-factor', GMfatt)
# ops.pattern('UniformExcitation', IDloadTag, GMdirection, '-accel', 20000) 

# # ops.wipeAnalysis()
# ops.constraints('Transformation')
# ops.numberer('RCM')
# ops.system('BandGeneral')
# #op.test('EnergyIncr', Tol, maxNumIter)
# #op.algorithm('ModifiedNewton')
# #NewmarkGamma = 0.5
# #NewmarkBeta = 0.25
# #op.integrator('Newmark', NewmarkGamma, NewmarkBeta)
# #op.analysis('Transient')


# #Nsteps =  int(TmaxAnalysis/ DtAnalysis)
# #
# #ok = op.analyze(Nsteps, DtAnalysis)

# tCurrent = ops.getTime()

# # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

# # test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 3:'EnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
# # algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 3:'ModifiedNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}
# test={1:'EnergyIncr'}
# algorithm={1:'ModifiedNewton'}
# # tFinal = nPts*dt

# tFinal = 60.0
# time = [tCurrent]
# u3 = [0.0]
# u4 = [0.0]
# ok = 0
# while tCurrent < tFinal:
# #    ok = op.analyze(1, .01)     
#     for i in test:
#         for j in algorithm: 
#             if j < 4:
#                 ops.algorithm(algorithm[j], '-initial')
                
#             else:
#                 ops.algorithm(algorithm[j])
#             while ok == 0 and tCurrent < tFinal:
                    
#                 ops.test(test[i], Tol, maxNumIter)        
#                 NewmarkGamma = 0.5
#                 NewmarkBeta = 0.25
#                 ops.integrator('Newmark', NewmarkGamma, NewmarkBeta)
#                 ops.analysis('Transient')
#                 ok = ops.analyze(1, .01)
                
#                 if ok == 0 :
#                     tCurrent = ops.getTime()                
#                     time.append(tCurrent)
#                     u3.append(ops.nodeDisp(10,1))
# #                     u4.append(ops.nodeDisp(4,1))
#                     print(test[i], algorithm[j], 'tCurrent=', tCurrent)

        
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8,8))
# plt.plot(time, u3)
# plt.ylabel('Horizontal Displacement of node 3 (in)')
# plt.xlabel('Time (s)')
# plt.savefig('Horizontal Disp at Node 3 vs time.jpeg', dpi = 500)
# plt.show()

# # plt.figure(figsize=(8,8))
# # plt.plot(time, u4)
# # plt.ylabel('Horizontal Displacement of node 4 (in)')
# # plt.xlabel('Time (s)')
# # plt.savefig('Horizontal Disp at Node 4 vs time.jpeg', dpi = 500)
# # plt.show() 


# ops.wipe()